Данный алгоритм принимает на вход невопросительные предложения из русскоязычной Википедии и преобразует их в общие вопросы (т.е. такие, на который ответом может быть "да" или "нет"). 

# Загрузка библиотек и модели

---



In [ ]:
from IPython.display import clear_output

In [ ]:
# stanza – языковая модель для морфологического и синтаксического парсинга
!pip install stanza
# pymorphy – морфологический анализатор для русского
!pip install pymorphy2
# nltk для операций над текстом (в основном, нужен для токенизации на предложения)
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

clear_output()

In [ ]:
# udapi

!pip install --upgrade git+https://github.com/udapi/udapi-python.git
import os
os.environ['PATH'] += ":" + os.path.join(os.environ['HOME'], ".local/bin")

from udapi.block.read.conllu import Conllu
from udapi.core.document import Document
from udapi.block.write.textmodetrees import TextModeTrees

from io import StringIO

clear_output()

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# загружаем модель для русского языка, инициализируем морф. анализатор

import stanza
from stanza.utils.conll import CoNLL
import pymorphy2
import re
import pandas as pd

morph = pymorphy2.MorphAnalyzer()
stanza_model = stanza.Pipeline('ru', processors='tokenize,pos,lemma,depparse')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


# Отбор необходимых узлов 

---

Приведены функции для нескольких случаев, которые, получая на вход дерево зависимостей, отбирают необходимые узлы, которые затем будут задействованы в генерации вопроса. 


In [ ]:
# функция для нахождения необходимых частей предложения 
def find_nodes_first(nodes):
    for node in nodes:
        if node.deprel == "root" and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin': # находим корень-глагол
            fin_verb = node
            for child in fin_verb.children:
                if child.deprel == 'nsubj' and child.upos in ['NOUN', 'PROPN']: # находим одушевленный субъект глагола
                    verb_subject = child
                elif child.deprel == 'obj' and child.upos in ['NOUN', 'PROPN']: # находим прямой объект глагола 
                    verb_object = child
    return fin_verb, verb_subject, verb_object 

In [ ]:
# функция для нахождения необходимых частей предложения 
def find_nodes_second(nodes):
    for node in nodes:
        if node.deprel == "root" and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin': # находим корень-глагол
            fin_verb = node
            for child in fin_verb.children:
                if child.deprel == 'nsubj' and child.upos in ['NOUN', 'PROPN']: # находим одушевленный субъект глагола
                    verb_subject = child
                elif child.deprel == 'obl' and child.upos in ['NOUN', 'PROPN']: # находим адъюнкт 
                    for obl_child in child.children:
                        if obl_child.deprel == 'case' and obl_child.ord == fin_verb.ord + 1:
                            oblique_object = child
                            oblique_object_prep = obl_child
    return fin_verb, verb_subject, oblique_object, oblique_object_prep 

# Создание вопросительной и отрицательной частиц

---

Ниже написаны функции, которые генерируют вопросительную частицу *ли*, частицу *не...ли*, отрицательную частицу *не*, а также сочетание наречия *правда* с частицей *ли* (*правда ли* ...)

В них учитываются правые и левые контексты узлов, к которым задается вопрос, и добавляются те или иные вопросительные компоненты. 


In [ ]:
# функция, создающая вопросительную частицу "ли"
# VP_focused=True , когда на первом месте стоит глагол
# NP_focused=True , когда на первом месте стоит любо субъект, либо объект, либо косвнный объект (адъюнкт)
# параметр transitivity различает переходные/неперходные глаголы (Если он указан  True, то у глагола точно есть объект, иначе - есть адъюнкт)
# subject_before_object - параметр, который позволяет различать, когда на первом месте субъект, а когда - объект (для переходных глаголов)
# subject_before_oblique - параметр, который позволяет различать, когда на первом месте субъект, а когда - адъюнкт (для непереходных глаголов)

def add_li_participle(node, VP_focused=False, XP_focused=False, transitivity=False, subject_before_object=False, subject_before_oblique=False):
    if VP_focused == True: 
        if node.deprel == 'root' and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin':
            if node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                datum = node.prev_node
                if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                    number = datum.prev_node
                    if number.prev_node is not None and number.prev_node.deprel == 'flat':
                        month = number.prev_node
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(month.prev_node)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(number)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                    flat_node = datum.prev_node
                    if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(flat_node.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(datum.prev_node)
                else:
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(datum)
            else:
                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                li_participle.shift_after_node(node)
    if XP_focused == True:
        if transitivity == True:
            if subject_before_object == True:
              if node.deprel == 'nsubj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                  if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                      char = node.prev_node
                      if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(char.prev_node)
                      else:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(node.prev_node)
                  elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                      mod = node.prev_node
                      if mod.prev_node is not None and mod.prev_node.deprel in ['amod', 'nmod']:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(mod.prev_node)
                      else:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(mod)
                  elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                      datum = node.prev_node
                      if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                          number = datum.prev_node
                          if number.prev_node is not None and number.prev_node.deprel == 'flat':
                              month = number.prev_node
                              li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                              li_participle.shift_after_node(month.prev_node)
                          else:
                              li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                              li_participle.shift_after_node(number)
                      elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                          flat_node = datum.prev_node
                          if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                              li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                              li_participle.shift_after_node(flat_node.prev_node)
                      elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(datum.prev_node)
                      else:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(datum)
                  else:
                      li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                      li_participle.shift_after_node(node)
            elif subject_before_object == False:
              if node.deprel == 'obj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                  if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                      char = node.prev_node
                      if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(char.prev_node)
                      else:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(node.prev_node)
                  elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                      mod = node.prev_node
                      if mod.prev_node is not None and mod.prev_node.deprel in ['amod', 'nmod']:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(mod.prev_node)
                      else:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(mod)
                  elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                      datum = node.prev_node
                      if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                          number = datum.prev_node
                          if number.prev_node is not None and number.prev_node.deprel == 'flat':
                              month = number.prev_node
                              li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                              li_participle.shift_after_node(month.prev_node)
                          else:
                              li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                              li_participle.shift_after_node(number)
                      elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                          flat_node = datum.prev_node
                          if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                              li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                              li_participle.shift_after_node(flat_node.prev_node)
                      elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(datum.prev_node)
                      else:
                          li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                          li_participle.shift_after_node(datum)
                  else:
                      li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                      li_participle.shift_after_node(node)
        elif transitivity == False:
            if subject_before_oblique == True:
                if node.deprel == 'nsubj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                    if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                        char = node.prev_node
                        if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_after_node(char.prev_node)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_after_node(node.prev_node)
                    elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                        mod = node.prev_node
                        if mod.prev_node is not None and mod.prev_node.deprel in ['amod', 'nmod']:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_after_node(mod.prev_node)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_after_node(mod)
                    elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                        datum = node.prev_node
                        if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                            number = datum.prev_node
                            if number.prev_node is not None and number.prev_node.deprel == 'flat':
                                month = number.prev_node
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_after_node(month.prev_node)
                            else:
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_after_node(number)
                        elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                            flat_node = datum.prev_node
                            if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_after_node(flat_node.prev_node)
                        elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_after_node(datum.prev_node)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_after_node(datum)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(node)
            elif subject_before_oblique == False:
                if node.deprel == 'obl' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                    if node.prev_node is not None and node.prev_node.deprel in ['nmod', 'det', 'amod']:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(node.prev_node)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(node)
  
    return li_participle

In [ ]:
# Функции для создания пар "Правда+ли"
# логика такая же, как и при добавлении отдельного ли, но добавляем еще и узел "правда"

def add_truth_node(node, VP_focused=False, XP_focused=False, transitivity=False, subject_before_object=False, subject_before_oblique=False): 
    if VP_focused == True:
        if node.deprel == 'root' and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin': 
            if node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                datum = node.prev_node
                if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                    number = datum.prev_node
                    if number.prev_node is not None and number.prev_node.deprel == 'flat':
                        month = number.prev_node
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(month.prev_node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
                    else:
                      li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                      li_participle.shift_before_node(number)
                      truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                      truth_node.shift_before_node(li_participle)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                    flat_node = datum.prev_node
                    if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(flat_node.prev_node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle) 
                elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_before_node(datum.prev_node)
                    truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                    truth_node.shift_before_node(li_participle)
                else:
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_before_node(datum)
                    truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                    truth_node.shift_before_node(li_participle)
            else:
                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                li_participle.shift_before_node(node)
                truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                truth_node.shift_before_node(li_participle)
    if XP_focused == True: 
        if transitivity == True:
            if subject_before_object == True:
              if node.deprel == 'nsubj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                    char = node.prev_node
                    if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(char.prev_node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(node.prev_node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
                elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                    mod = node.prev_node
                    if mod.prev_node is not None and mod.prev_node.deprel in ['amod', 'nmod']:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(mod.prev_node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(mod)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
                elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                    datum = node.prev_node
                    if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                        number = datum.prev_node
                        if number.prev_node is not None and number.prev_node.deprel == 'flat':
                            month = number.prev_node
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(month.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(number)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                    elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                        flat_node = datum.prev_node
                        if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(flat_node.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle) 
                    elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(datum.prev_node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(datum)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
                else:
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_before_node(node)
                    truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                    truth_node.shift_before_node(li_participle) 
            elif subject_before_object == False:
                if node.deprel == 'obj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                    if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                        char = node.prev_node
                        if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(char.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(node.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                    elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                        mod = node.prev_node
                        if mod.prev_node is not None and mod.prev_node.deprel in ['amod', 'nmod']:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(mod.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(mod)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                    elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                        datum = node.prev_node
                        if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                            number = datum.prev_node
                            if number.prev_node is not None and number.prev_node.deprel == 'flat':
                                month = number.prev_node
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_before_node(month.prev_node)
                                truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                                truth_node.shift_before_node(li_participle)
                            else:
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_before_node(number)
                                truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                                truth_node.shift_before_node(li_participle)
                        elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                            flat_node = datum.prev_node
                            if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_before_node(flat_node.prev_node)
                                truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                                truth_node.shift_before_node(li_participle) 
                        elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(datum.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(datum)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
        elif transitivity == False:
            if subject_before_oblique == True:
                if node.deprel == 'nsubj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                    if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                        char = node.prev_node
                        if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(char.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(node.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                    elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                        mod = node.prev_node
                        if mod.prev_node is not None and mod.prev_node.deprel in ['amod', 'nmod']:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(mod.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(mod)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                    elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                        datum = node.prev_node
                        if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                            number = datum.prev_node
                            if number.prev_node is not None and number.prev_node.deprel == 'flat':
                                month = number.prev_node
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_before_node(month.prev_node)
                                truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                                truth_node.shift_before_node(li_participle)
                            else:
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_before_node(number)
                                truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                                truth_node.shift_before_node(li_participle)
                        elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                            flat_node = datum.prev_node
                            if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                                li_participle.shift_before_node(flat_node.prev_node)
                                truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                                truth_node.shift_before_node(li_participle) 
                        elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(datum.prev_node)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                        else:
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(datum)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
                    else:
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_before_node(node)
                        truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                        truth_node.shift_before_node(li_participle)
            elif subject_before_oblique == False:
                if node.deprel == 'obl' and node.upos in ['NOUN', 'PROPN', 'PRON']:
                    for child in node.children:
                        if child.deprel == 'case':
                            adjunct_prep = child
                            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                            li_participle.shift_before_node(adjunct_prep)
                            truth_node = node.create_child(form='правда', lemma='правда', upos='ADV', feats='Degree=Pos', deprel='advmod')
                            truth_node.shift_before_node(li_participle)
  
    return li_participle, truth_node

In [ ]:
# Функция, которая добавляет отрицательную частицу к опредленному слову в предложении 

# Verb_negation=True для случаев типа "Вася не прочитал книгу"
# Subject_negation=True для случаев типа "Не Вася прочитал книгу"
# Object_negation=True для случаев типа "Вася прочитал не книгу"
# Adjunct_negation=True для случаев типа "Не в школу Вася ходит?"

def add_negative_participle(node, Verb_negation=False, Subject_negation=False, Object_negation=False, Adjunct_negation=False):
    if Verb_negation == True:
        if node.deprel == 'root' and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin':
            ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
            ne_participle.shift_before_node(node)
    if Subject_negation == True:
        if node.deprel == 'nsubj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
            if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                char = node.prev_node
                if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char)
            elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                mod = node.prev_node
                if mod.prev_node is not None and mod.prev_node.deprel == 'amod':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod)
            elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                datum = node.prev_node
                if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                    number = datum.prev_node
                    if number.prev_node is not None and number.prev_node.deprel == 'flat':
                        month = number.prev_node
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(month.prev_node)
                    else:
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(datum.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                    flat_node = datum.prev_node
                    if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(flat_node.prev_node)
                    elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(datum.prev_node)
                    else:
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(datum)  
            else:
                ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                ne_participle.shift_before_node(node)
    if Object_negation == True:
        if node.deprel == 'obj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
            if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                char = node.prev_node
                if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char)
            elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                mod = node.prev_node
                if mod.prev_node is not None and mod.prev_node.deprel == 'amod':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod)
            elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                datum = node.prev_node
                if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                    number = datum.prev_node
                    if number.prev_node is not None and number.prev_node.deprel == 'flat':
                        month = number.prev_node
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(month.prev_node)
                    else:
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(datum.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                    flat_node = datum.prev_node
                    if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(flat_node.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum)
            else:
                ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                ne_participle.shift_before_node(node)
    if Adjunct_negation == True:
        if node.deprel == 'obl' and node.upos in ['NOUN', 'PROPN', 'PRON']:
            for child in node.children:
                if child.deprel == 'case' and child.ord < node.ord:
                    adjunct_prep = child
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(adjunct_prep)
  
    return ne_participle

In [ ]:
# Функция, которая добавляет частицу не...ли 

def add_ne_li_participle(node, Verb_negation=False, Subject_negation=False, Object_negation=False, Adjunct_negation=False):
    if Verb_negation == True:
        if node.deprel == 'root' and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin':
            if node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                datum = node.prev_node
                if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                    number = datum.prev_node
                    if number.prev_node is not None and number.prev_node.deprel == 'flat':
                        month = number.prev_node
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(month.prev_node)
                    else:
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(datum.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                    flat_node = datum.prev_node
                    if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(flat_node.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum)  
            else:
                ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                ne_participle.shift_before_node(node)
            li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
            li_participle.shift_after_node(node) 
    if Subject_negation == True:
        if node.deprel == 'nsubj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
            if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                char = node.prev_node
                if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char.prev_node)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(char.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(char)
            elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                mod = node.prev_node
                if mod.prev_node is not None and mod.prev_node.deprel == 'amod':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod.prev_node)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(mod.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(mod)
            elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                datum = node.prev_node
                if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                    number = datum.prev_node
                    if number.prev_node is not None and number.prev_node.deprel == 'flat':
                        month = number.prev_node
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(month.prev_node)
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(month.prev_node)
                    else:
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(datum.prev_node)
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(datum.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                    flat_node = datum.prev_node
                    if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(flat_node.prev_node)
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(flat_node.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum.prev_node)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(datum.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum) 
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(datum) 
            else:
                ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                ne_participle.shift_before_node(node)
                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                li_participle.shift_after_node(node) 
    if Object_negation == True:
        if node.deprel == 'obj' and node.upos in ['NOUN', 'PROPN', 'PRON']:
            if node.prev_node is not None and node.prev_node.deprel in ['amod', 'det']:
                char = node.prev_node
                if char.prev_node is not None and char.prev_node.deprel in ['amod', 'det']:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char.prev_node)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(char.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(char)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(char)
            elif node.prev_node is not None and node.prev_node.deprel == 'nmod':
                mod = node.prev_node
                if mod.prev_node is not None and mod.prev_node.deprel == 'amod':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod.prev_node)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(mod.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(mod)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(mod)
            elif node.prev_node is not None and node.prev_node.deprel in ['nmod', 'flat', 'nummod']:
                datum = node.prev_node
                if datum.prev_node is not None and datum.prev_node.deprel == 'nummod':
                    number = datum.prev_node
                    if number.prev_node is not None and number.prev_node.deprel == 'flat':
                        month = number.prev_node
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(month.prev_node)
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(month.prev_node)
                    else:
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(datum.prev_node)
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(datum.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'flat': 
                    flat_node = datum.prev_node
                    if flat_node.prev_node is not None and flat_node.prev_node.deprel == 'nummod':
                        ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                        ne_participle.shift_before_node(flat_node.prev_node)
                        li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                        li_participle.shift_after_node(flat_node.prev_node)
                elif datum.prev_node is not None and datum.prev_node.deprel == 'case':
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum.prev_node)
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(datum.prev_node)
                else:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(datum) 
                    li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                    li_participle.shift_after_node(datum) 
            else:
                ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                ne_participle.shift_before_node(node)
                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                li_participle.shift_after_node(node) 
    if Adjunct_negation == True:
        if node.deprel == 'obl' and node.upos in ['NOUN', 'PROPN', 'PRON']:
            for obl_child in node.children:
                if obl_child.deprel == 'case' and obl_child.ord < node.ord:
                    ne_participle = node.create_child(form='не', lemma='не', upos='PART', deprel='advmod')
                    ne_participle.shift_before_node(obl_child)
            if node.prev_node is not None and node.prev_node.deprel in ['nmod', 'det', 'amod']:
                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                li_participle.shift_after_node(node.prev_node)
            else:
                li_participle = node.create_child(form='ли', lemma='ли', upos='PART', deprel='advmod')
                li_participle.shift_after_node(node)
          
    return ne_participle, li_participle

# Генерация вопросительных предложений

---

При генерации считается, что помимо главных членов предложения, чей порядок слов SVO, имеются также и другие составляющие, которые оказывают значительную роль на семантику вопроса. В числе этих значимых членов предложения находятся прилагательные, числовые и именные модификаторы, непрямой объект и тд. Они также учитываются, чтобы сгенерируемый вопрос был не только правильным синтаксически, но и семантически. 

---

Сначала идет 36 типов вопросов с ответом "да", затем 36 типов вопросов с ответом "нет".


## Генерация вопросов с ответом "да"

---

36 типов вопросов, построенные с частицами *ли*, *не...ли*, а также с наречием *правда* и частицей *ли* (правда ли...)

In [ ]:
# функция для создания вопроса типа "V ли Subject Obj ?"
def yes_first_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes) 
    if verb_subject.prev_node.deprel == 'amod':
        fin_verb.shift_before_node(verb_subject.prev_node, without_children=True)
        li_participle = add_li_participle(fin_verb, VP_focused=True)
    else:
        fin_verb.shift_before_node(verb_subject, without_children=True)
        li_participle = add_li_participle(fin_verb, VP_focused=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V ли Obj Subject ?"
def yes_second_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    if verb_object.next_node is not None and verb_object.next_node.deprel == 'nmod':
        obj_modifire = verb_object.next_node
        if obj_modifire.next_node is not None and obj_modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(obj_modifire, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'case':
        preposition = verb_object.next_node
        if preposition.next_node.deprel in ['nmod', 'amod']:
            modifire = preposition.next_node 
            if modifire.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(modifire.next_node, without_children=False)
            else:
                verb_subject.shift_after_node(modifire, without_children=False)
    else:
        verb_subject.shift_after_node(verb_object, without_children=False)
    li_participle = add_li_participle(fin_verb, VP_focused=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject ли V Obj ?"
def yes_third_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    li_participle = add_li_participle(verb_subject, XP_focused=True, transitivity=True, subject_before_object=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject ли Obj V ?"
def yes_fourth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    li_participle = add_li_participle(verb_subject, XP_focused=True, transitivity=True, subject_before_object=True)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Obj ли Subject V ?"
def yes_fifth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    li_participle = add_li_participle(verb_object, XP_focused=True, transitivity=True, subject_before_object=False)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        verb_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        verb_object.shift_before_node(verb_subject, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Obj ли V Subject ?"
def yes_sixth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    li_participle = add_li_participle(verb_object, XP_focused=True, transitivity=True, subject_before_object=False)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Subject Verb Obj ?"
def yes_seventh_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    li_participle, truth_node = add_truth_node(verb_subject, XP_focused=True, transitivity=True, subject_before_object=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Subject Obj Verb ?"
def yes_eighth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    li_participle, truth_node = add_truth_node(verb_subject, XP_focused=True, transitivity=True, subject_before_object=True)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Obj Verb Subject ?"
def yes_ninth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    li_participle, truth_node = add_truth_node(verb_object, XP_focused=True, transitivity=True, subject_before_object=False)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Obj Subject Verb ?"
def yes_tenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    li_participle, truth_node = add_truth_node(verb_object, XP_focused=True, transitivity=True, subject_before_object=False)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod': 
        verb_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        verb_object.shift_before_node(verb_subject, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Verb Subject Obj?"
def yes_eleventh_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    li_participle, truth_node = add_truth_node(fin_verb, VP_focused=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Verb Obj Subject ?"
def yes_twelfth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    if verb_object.next_node is not None and verb_object.next_node.deprel == 'nmod':
        obj_modifire = verb_object.next_node
        if obj_modifire.next_node is not None and obj_modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(obj_modifire, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'amod':
            obj_adjective = verb_object.next_node
            if obj_adjective.next_node is not None and obj_adjective.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(obj_adjective.next_node, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'case':
        preposition = verb_object.next_node
        if preposition.next_node.deprel in ['nmod', 'amod']:
            modifire = preposition.next_node 
            if modifire.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(modifire.next_node, without_children=False)
            else:
                verb_subject.shift_after_node(modifire, without_children=False)
    else:
        verb_subject.shift_after_node(verb_object, without_children=False)
    li_participle, truth_node = add_truth_node(fin_verb, VP_focused=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V ли S Preposition X ?"
def yes_thirteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    li_participle = add_li_participle(fin_verb, VP_focused=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V ли preposition X S ?"
def yes_fourteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    li_participle = add_li_participle(fin_verb, VP_focused=True)
    if oblique_object.next_node is not None and oblique_object.next_node.deprel == 'nmod':
        verb_subject.shift_after_node(oblique_object.next_node, without_children=False)
    else:
        verb_subject.shift_after_node(oblique_object, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Preposition X ли Subject Verb ?"
def yes_fifteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    li_participle = add_li_participle(oblique_object, XP_focused=True, transitivity=False, subject_before_oblique=False)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        oblique_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        oblique_object.shift_before_node(verb_subject, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Preposition X ли Verb Subject ?"
def yes_sixteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    li_participle = add_li_participle(oblique_object, XP_focused=True, transitivity=False, subject_before_oblique=False)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject ли Verb Preposition X ?"
def yes_seventeenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    li_participle = add_li_participle(verb_subject, XP_focused=True, transitivity=False, subject_before_oblique=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject ли Preposition X Verb ?"
def yes_eighteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    li_participle = add_li_participle(verb_subject, XP_focused=True, transitivity=False, subject_before_oblique=True)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Subject V Prep X ?"
def yes_nineteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    li_participle, truth_node = add_truth_node(verb_subject, XP_focused=True, transitivity=False, subject_before_oblique=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Subject Prep X V ?"
def yes_twentieth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    li_participle, truth_node = add_truth_node(verb_subject, XP_focused=True, transitivity=False, subject_before_oblique=True)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли V Subject Prep X ?"
def yes_twenty_first_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    li_participle, truth_node = add_truth_node(fin_verb, VP_focused=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли V Prep X Subject ?"
def yes_twenty_second_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    if oblique_object.next_node is not None and oblique_object.next_node.deprel == 'nmod':
        modifire = oblique_object.next_node
        if modifire.next_node is not None and modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(modifire.next_node, without_children=False)
        elif modifire.next_node is not None and modifire.next_node.deprel == 'amod':
            adjective = modifire.next_node
            if adjective.next_node is not None and adjective.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(adjective.next_node, without_children=False)
            else:
                verb_subject.shift_after_node(modifire, without_children=False)
    else:
        verb_subject.shift_after_node(oblique_object, without_children=False)
    li_participle, truth_node = add_truth_node(fin_verb, VP_focused=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Prep Oblique Verb Subject ?"
def yes_twenty_third_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    li_participle, truth_node = add_truth_node(oblique_object, XP_focused=True, transitivity=False, subject_before_oblique=False)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Правда ли Prep Oblique Subject V ?"
def yes_twenty_fourth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    li_participle, truth_node = add_truth_node(oblique_object, XP_focused=True, transitivity=False, subject_before_oblique=False)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        oblique_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        oblique_object.shift_before_node(verb_subject, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Subject ли Verb Object ?" 
def yes_twenty_fifth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle, li_participle = add_ne_li_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Subject ли Object Verb ?" 
def yes_twenty_sixth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle, li_participle = add_ne_li_participle(verb_subject, Subject_negation=True)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Verb ли Subject Object ?" 
def yes_twenty_seventh_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle, li_participle = add_ne_li_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Verb ли Object Subject ?" 
def yes_twenty_eighth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    if verb_object.next_node is not None and verb_object.next_node.deprel == 'nmod':
        obj_modifire = verb_object.next_node
        if obj_modifire.next_node is not None and obj_modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(obj_modifire, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'amod':
        obj_adjective = verb_object.next_node
        if obj_adjective.next_node is not None and obj_adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_adjective.next_node, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'case':
        preposition = verb_object.next_node
        if preposition.next_node.deprel in ['nmod', 'amod']:
            modifire = preposition.next_node 
            if modifire.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(modifire.next_node, without_children=False)
            else:
                verb_subject.shift_after_node(modifire, without_children=False)
    else:
        verb_subject.shift_after_node(verb_object, without_children=False)
    ne_participle, li_participle = add_ne_li_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Object ли Subject Verb ?" 
def yes_twenty_ninth_type(sentence):
  tree = Conllu().read_tree_from_lines(sentence)
  nodes = tree.descendants
  fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
  ne_participle, li_participle = add_ne_li_participle(verb_object, Object_negation=True)
  if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
    verb_object.shift_before_node(verb_subject.prev_node, without_children=False)
  else:
    verb_object.shift_before_node(verb_subject, without_children=False)
  for node in nodes: 
    if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
      node.remove()
    elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
      node.remove()
    elif node.deprel == 'flat' and node.parent.upos != 'NUM':
      node.remove()
    elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
      node.remove()
  question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
  return tree

In [ ]:
# функция для создания вопроса типа "Не Object ли Verb Subject ?" 
def yes_thirtieth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle, li_participle = add_ne_li_participle(verb_object, Object_negation=True)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Verb ли Subject Preposition Oblique_Object ?" 
def yes_thirty_first_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle, li_participle = add_ne_li_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Verb ли Preposition Oblique_Object Subject ?" 
def yes_thirty_second_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    if oblique_object.next_node is not None and oblique_object.next_node.deprel == 'nmod':
        modifire = oblique_object.next_node 
        verb_subject.shift_after_node(modifire, without_children=False)
    elif oblique_object.next_node is not None and oblique_object.next_node.deprel == 'amod':
        adjective = oblique_object.next_node 
        if adjective.next_node is not None and adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(adjective.next_node, without_children=False)
    else:
        verb_subject.shift_after_node(oblique_object, without_children=False)
    ne_participle, li_participle = add_ne_li_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Subject ли Verb Preposition Oblique_Object ?" 
def yes_thirty_third_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle, li_participle = add_ne_li_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Subject ли Preposition Oblique_Object Verb ?" 
def yes_thirty_fourth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    ne_participle, li_participle = add_ne_li_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Preposition Oblique_Object ли Subject Verb ?" 
def yes_thirty_fifth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle, li_participle = add_ne_li_participle(oblique_object, Adjunct_negation=True)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        oblique_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        oblique_object.shift_before_node(verb_subject, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Preposition Oblique_Object ли Verb Subject ?" 
def yes_thirty_sixth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle, li_participle = add_ne_li_participle(oblique_object, Adjunct_negation=True)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

## Генерация вопросов с ответом "нет"

---

36 типов вопросов, построенных с частицей *не*

In [ ]:
# функция для создания вопроса типа "Не Subject V Object ?"
def no_first_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject не V Object ?"
def no_second_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject V не Object ?"
def no_third_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(verb_object, Object_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Subject Object V ?"
def no_fourth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_object.shift_before_node(fin_verb, without_children=False)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject не Object V ?"
def no_fifth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(verb_object, Object_negation=True)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject Object не V?"
def no_sixth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    verb_object.shift_before_node(ne_participle, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не V Subject Object?"
def no_seventh_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V не Subject Object?"
def no_eighth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V Subject не Object?"
def no_ninth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle = add_negative_participle(verb_object, Object_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не V Object Subject ?"
def no_tenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    if verb_object.next_node is not None and verb_object.next_node.deprel == 'nmod':
        obj_modifire = verb_object.next_node
        if obj_modifire.next_node is not None and obj_modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(obj_modifire, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'amod':
        obj_adjective = verb_object.next_node
        if obj_adjective.next_node is not None and obj_adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_adjective.next_node, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'case':
        preposition = verb_object.next_node
        if preposition.next_node.deprel in ['nmod', 'amod']:
            modifire = preposition.next_node 
            if modifire.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(modifire.next_node, without_children=False)
            else:
                verb_subject.shift_after_node(modifire, without_children=False)
    else:
        verb_subject.shift_after_node(verb_object, without_children=False)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V не Object Subject ?"
def no_eleventh_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    if verb_object.next_node is not None and verb_object.next_node.deprel == 'nmod':
        obj_modifire = verb_object.next_node
        if obj_modifire.next_node is not None and obj_modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(obj_modifire, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'amod':
        obj_adjective = verb_object.next_node
        if obj_adjective.next_node is not None and obj_adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_adjective.next_node, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'case':
        preposition = verb_object.next_node
        if preposition.next_node.deprel in ['nmod', 'amod']:
            modifire = preposition.next_node 
            if modifire.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(modifire.next_node, without_children=False)
            else:
                verb_subject.shift_after_node(modifire, without_children=False)
    else:
        verb_subject.shift_after_node(verb_object, without_children=False)
    ne_participle = add_negative_participle(verb_object, Object_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V Object не Subject ?"
def no_twelfth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    if verb_object.next_node is not None and verb_object.next_node.deprel == 'nmod':
        obj_modifire = verb_object.next_node
        if obj_modifire.next_node is not None and obj_modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(obj_modifire, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'amod':
        obj_adjective = verb_object.next_node
        if obj_adjective.next_node is not None and obj_adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(obj_adjective.next_node, without_children=False)
    elif verb_object.next_node is not None and verb_object.next_node.deprel == 'case':
        preposition = verb_object.next_node
        if preposition.next_node.deprel in ['nmod', 'amod']:
            modifire = preposition.next_node 
            if modifire.next_node.deprel == 'nmod':
                verb_subject.shift_after_node(modifire.next_node, without_children=False)
            else:
                verb_subject.shift_after_node(modifire, without_children=False)
    else:
        verb_subject.shift_after_node(verb_object, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Object V Subject ?"
def no_thirteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False) 
    ne_participle = add_negative_participle(verb_object, Object_negation=True)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Object не V Subject ?"
def no_fourteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    verb_object.shift_before_node(fin_verb, without_children=False)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Object V не Subject ?"
def no_fifteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    verb_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Object Subject V?"
def no_sixteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    ne_participle = add_negative_participle(verb_object, Object_negation=True)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        verb_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        verb_object.shift_before_node(verb_subject, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Object не Subject V?"
def no_seventeenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        verb_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        verb_object.shift_before_node(verb_subject, without_children=False)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Object Subject не V?"
def no_eighteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, verb_object = find_nodes_first(nodes)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        verb_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        verb_object.shift_before_node(verb_subject, without_children=False)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obj', 'iobj', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'flat:name', 'flat', 'xcomp']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
              node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Subject V Prep Oblique?"
def no_nineteenth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
      if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
          node.remove()
      elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
          node.remove()
      elif node.deprel == 'fixed' and node.form != 'течение':
          node.remove()
      elif node.deprel == 'flat' and node.parent.upos != 'NUM':
          node.remove()
      elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
          node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject не V Prep Oblique?"
def no_twentieth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
      if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
          node.remove()
      elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
          node.remove()
      elif node.deprel == 'fixed' and node.form != 'течение':
          node.remove()
      elif node.deprel == 'flat' and node.parent.upos != 'NUM':
          node.remove()
      elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
          node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject V не Prep Oblique?"
def no_twenty_first_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(oblique_object, Adjunct_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Subject Prep Oblique V?"
def no_twenty_second_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    oblique_object.shift_before_node(fin_verb, without_children=True)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject не Prep Oblique V?"
def no_twenty_third_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(oblique_object, Adjunct_negation=True)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Subject Prep Oblique не V?"
def no_twenty_fourth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    oblique_object.shift_before_node(ne_participle, without_children=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не V Subject Prep Oblique?"
def no_twenty_fifth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V не Subject Prep Oblique?"
def no_twenty_sixth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V Subject не Prep Oblique?"
def no_twenty_seventh_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle = add_negative_participle(oblique_object, Adjunct_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не V Prep Oblique Subject?"
def no_twenty_eighth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    if oblique_object.next_node is not None and oblique_object.next_node.deprel == 'nmod':
        modifire = oblique_object.next_node 
        if modifire.next_node is not None and modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(modifire, without_children=False)
    elif oblique_object.next_node is not None and oblique_object.next_node.deprel == 'amod':
        adjective = oblique_object.next_node 
        if adjective.next_node is not None and adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(adjective.next_node, without_children=False)
    else:
        verb_subject.shift_after_node(oblique_object, without_children=False)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V не Prep Oblique Subject?"
def no_twenty_ninth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    if oblique_object.next_node is not None and oblique_object.next_node.deprel == 'nmod':
        modifire = oblique_object.next_node 
        if modifire.next_node is not None and modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(modifire, without_children=False)
    elif oblique_object.next_node is not None and oblique_object.next_node.deprel == 'amod':
        adjective = oblique_object.next_node 
        if adjective.next_node is not None and adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(adjective.next_node, without_children=False)
    else:
        verb_subject.shift_after_node(oblique_object, without_children=False)
    ne_participle = add_negative_participle(oblique_object, Adjunct_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "V Prep Oblique не Subject?"
def no_thirtieth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    if oblique_object.next_node is not None and oblique_object.next_node.deprel == 'nmod':
        modifire = oblique_object.next_node 
        if modifire.next_node is not None and modifire.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(modifire.next_node, without_children=False)
        else:
            verb_subject.shift_after_node(modifire, without_children=False)
    elif oblique_object.next_node is not None and oblique_object.next_node.deprel == 'amod':
        adjective = oblique_object.next_node 
        if adjective.next_node is not None and adjective.next_node.deprel == 'nmod':
            verb_subject.shift_after_node(adjective.next_node, without_children=False)
    else:
        verb_subject.shift_after_node(oblique_object, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Prep Oblique Subject V?"
def no_thirty_first_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(oblique_object, Adjunct_negation=True)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        oblique_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        oblique_object.shift_before_node(verb_subject, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Prep Oblique не Subject V?"
def no_thirty_second_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    oblique_object.shift_before_node(ne_participle, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Prep Oblique Subject не V?"
def no_thirty_third_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    if verb_subject.prev_node is not None and verb_subject.prev_node.deprel == 'amod':
        oblique_object.shift_before_node(verb_subject.prev_node, without_children=False)
    else:
        oblique_object.shift_before_node(verb_subject, without_children=False)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Не Prep Oblique V Subject?"
def no_thirty_fourth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    ne_participle = add_negative_participle(oblique_object, Adjunct_negation=True)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
            node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
            node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
            node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
            node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
            node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Prep Oblique не V Subject?"
def no_thirty_fifth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    ne_participle = add_negative_participle(fin_verb, Verb_negation=True)
    for node in nodes: 
          if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
              node.remove()
          elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
              node.remove()
          elif node.deprel == 'fixed' and node.form != 'течение':
              node.remove()
          elif node.deprel == 'flat' and node.parent.upos != 'NUM':
              node.remove()
          elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
              node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

In [ ]:
# функция для создания вопроса типа "Prep Oblique V не Subject?"
def no_thirty_sixth_type(sentence):
    tree = Conllu().read_tree_from_lines(sentence)
    nodes = tree.descendants
    fin_verb, verb_subject, oblique_object, oblique_object_prep = find_nodes_second(nodes)
    ne_participle = add_negative_participle(verb_subject, Subject_negation=True)
    verb_subject.shift_after_node(fin_verb, without_children=False)
    oblique_object.shift_before_node(fin_verb, without_children=False)
    for node in nodes: 
        if node.deprel not in ['root', 'nsubj', 'obl', 'amod', 'nmod', 'case', 'nummod', 'nummod:gov', 'det', 'appos', 'fixed', 'flat:name', 'flat', 'xcomp', 'obj']:
          node.remove()
        elif node.deprel == 'appos' and node.parent.deprel not in ['nsubj', 'nmod', 'obj']:
          node.remove()
        elif node.deprel == 'fixed' and node.form != 'течение':
          node.remove()
        elif node.deprel == 'flat' and node.parent.upos != 'NUM':
          node.remove()
        elif node.deprel == 'xcomp' and node.upos != 'VERB' and node.parent.deprel != 'root':
          node.remove()
    question_mark = fin_verb.create_child(form="?", lemma="?", upos="PUNCT", deprel='punct')
    return tree

# Чтение файла с данными из Википедии и отбор подходящих предложений

## Чтение файла с предложениями для отбора и обучение модели syntagrus на этом файле

---

Ниже из файла **wiki_sents_clear.txt** отбираются подходящие по синтаксической структуре предложения, которые затем подаются на вход модели, преобразовывающей их в деверья зависимостей. Далее для каждого отдельного типа предложения строится тип вопроса, определяемый случайным образом. 



In [ ]:
MY_DIR = "/content/drive/MyDrive/Diploma 2.0/parsed wiki 2023/"

In [ ]:
# функция для чтения файла
def read_infile(infile):
  with open(infile, 'r', encoding='utf8') as fin:
    wiki_sents = []
    for line in fin:
      line = line.strip()
      if line == "":
        pass
      else:
        wiki_sents.append(line)
  return wiki_sents

In [ ]:
wiki_file = os.path.join(MY_DIR, "wiki_sentences_2023.txt")
wiki_data = read_infile(wiki_file)

In [ ]:
wiki_small_data = read_infile("/content/new_wiki_texts.txt")

In [ ]:
len(wiki_small_data) # общая длина файла с предложениями из статей Википедии 

770393

In [ ]:
new_wiki = []
for wiki in wiki_data: 
    wiki = re.sub('\.', '. ', wiki)
    wiki = re.sub('  ', ' ', wiki)
    new_wiki.append(wiki)

In [ ]:
with open("new_wiki_texts_big.txt" "w") as fin:
    for elem in new_wiki:
        fin.write(elem + '\n')

In [ ]:
len(new_wiki)

5000000

In [ ]:
wiki_upd_small = []
for text in new_wiki[:500000]: 
    sents = nltk.sent_tokenize(text, "russian")
    wiki_upd_small = wiki_upd_small + sents

In [ ]:
 len(wiki_upd_small)

770393

In [ ]:
with open("new_wiki_texts.txt", "w") as fin:
    for elem in wiki_upd_small:
        fin.write(elem + '\n')

## Отбор предложений из текстов по их синтаксической структуре

In [ ]:
# функция, отбирающая предложения по синтаксическому шаблону (структура предложений: S V DO) (без прилагательных и модификаторов)
def search_sentences_first(sentences, suitable_sentences):
    for sentence in sentences:
        doc = stanza_model(sentence)
        conll_sent = CoNLL.doc2conll(doc)[0]
        tree = Conllu().read_tree_from_lines(conll_sent)
        nodes = tree.descendants
        found_nodes = []
        for node in nodes:
            if node.deprel == 'root' and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin': 
                fin_verb = node
                found_nodes.append('verb')
                for child in fin_verb.children:
                    if child.upos in ['NOUN', 'PROPN'] and child.deprel == 'nsubj' and child.ord < fin_verb.ord: 
                        found_nodes.append('subject')
                    elif child.upos in ['NOUN', 'PROPN'] and child.deprel == 'obj' and child.ord > fin_verb.ord:
                        verb_object = child
                        found_nodes.append('direct_object')
        if all(elem in found_nodes for elem in ['verb', 'subject', 'direct_object']):
            suitable_sentences.append((sentence, conll_sent))

In [ ]:
# функция, отбирающая предложения по синтаксическому шаблону (структура: S V Prep Obl) -> отбираем только неперходные глаголы (нет прямого объекта)
def search_sentences_second(sentences, suitable_sentences):
    for sentence in sentences:
        doc = stanza_model(sentence)
        conll_sent = CoNLL.doc2conll(doc)[0]
        tree = Conllu().read_tree_from_lines(conll_sent)
        nodes = tree.descendants
        found_nodes = []
        for node in nodes:
            if node.deprel == 'root' and node.upos == 'VERB' and node.feats['VerbForm'] == 'Fin': 
                fin_verb = node
                found_nodes.append('verb')
                for child in fin_verb.children:
                    if child.upos in ['NOUN', 'PROPN'] and child.deprel == 'nsubj' and child.ord < fin_verb.ord: 
                        if child.next_node.form != 'и':
                            found_nodes.append('subject')
                    elif child.upos in ['NOUN', 'PROPN'] and child.deprel == 'obl' and child.ord > fin_verb.ord:
                        oblique = child
                        for oblique_child in oblique.children:
                            if oblique_child.deprel == 'case' and oblique_child.ord == fin_verb.ord + 1:
                                found_nodes.append('oblique_object')
                                found_nodes.append('oblique_object_prep')
        if all(elem in found_nodes for elem in ['verb', 'subject', 'oblique_object', 'oblique_object_prep']):
            suitable_sentences.append((sentence, conll_sent))

In [ ]:
selected_sentences_first = []
selected_sentences_second = []

In [ ]:
# Предложения с синтаксической структурой вида "Subject Verb Object"
search_sentences_first(wiki_small_data[200000:], selected_sentences_first)
selected_first_str, selected_first_conll = [elem[0] for elem in selected_sentences_first], [elem[1] for elem in selected_sentences_first]

In [ ]:
# Предложения с синтаксической структурой вида "Subject Verb Prep Oblique"
search_sentences_second(wiki_small_data[200000:], selected_sentences_second)
selected_second_str, selected_second_conll = [elem[0] for elem in selected_sentences_second], [elem[1] for elem in selected_sentences_second]

In [ ]:
len(selected_sentences_first)

2514

In [ ]:
len(selected_sentences_second)

2211

In [ ]:
selected_first_str, selected_first_conll = [elem[0] for elem in selected_sentences_first], [elem[1] for elem in selected_sentences_first]

In [ ]:
selected_second_str, selected_second_conll = [elem[0] for elem in selected_sentences_second], [elem[1] for elem in selected_sentences_second]

In [ ]:
selected_second_str, selected_second_conll = [elem[0] for elem in selected_sentences_second], [elem[1] for elem in selected_sentences_second]

In [ ]:
# Сохраним все отобранные предложения (на всякий случай)
with open("/content/drive/MyDrive/Diploma 2.0/DaNetQA 2.0/selected_sentences_first_fin.txt", 'w', encoding='utf8') as first:
    for sentence in selected_first_str: 
        first.write(sentence + '\n')

with open("/content/drive/MyDrive/Diploma 2.0/DaNetQA 2.0/selected_sentences_second_fin.txt", 'w', encoding='utf8') as second:
    for sentence in selected_second_str: 
        second.write(sentence + '\n')

# также запишем все предложения и в формате conllu
with open("/content/drive/MyDrive/Diploma 2.0/DaNetQA 2.0/selected_first_parsed_fin.conll", 'w', encoding='utf8') as first_conll:
    for i, sentence in enumerate(selected_first_conll): 
        new_id = re.sub('sent_id = 0', f'sent_id = {i}', sentence[1])
        sentence.pop(1)
        sentence.insert(1, new_id) 
        for line in sentence: 
            first_conll.write(line + '\n')
        first_conll.write('\n')

with open("/content/drive/MyDrive/Diploma 2.0/DaNetQA 2.0/selected_second_parsed_fin.conll", 'w', encoding='utf8') as second_conll:
    for i, sentence in enumerate(selected_second_conll): 
        new_id = re.sub('sent_id = 0', f'sent_id = {i}', sentence[1])
        sentence.pop(1)
        sentence.insert(1, new_id) 
        for line in sentence: 
            second_conll.write(line + '\n')
        second_conll.write('\n')

# Генерация вопросительных предложений

In [ ]:
def read_conll_to_list_lists(file):
    wiki_lists = []
    with open(file, 'r') as fin:
        full_data = fin.read()
        texts = full_data.split('\n\n')
        for text in texts:
            text_lst = text.split('\n')
            wiki_lists.append(text_lst)
    return wiki_lists

wiki_first_type = read_conll_to_list_lists("/content/drive/MyDrive/Diploma 2.0/DaNetQA 2.0/selected_first_parsed_fin.conll")
wiki_second_type = read_conll_to_list_lists("/content/drive/MyDrive/Diploma 2.0/DaNetQA 2.0/selected_second_parsed_fin.conll")

In [ ]:
print(len(wiki_first_type))
print(len(wiki_second_type))

2515
2212


In [ ]:
2515//2

1257

In [ ]:
2212//2

1106

In [ ]:
len(wiki_first_type[:1256])

1256

In [ ]:
# Делим оба типа предложений на две части: 
# на одной будем генерировать вопросы с положительным ответом
# на другой – вопросы с отрицательным ответом

wiki_first, wiki_second = wiki_first_type[:1256], wiki_second_type[:1106]
wiki_third, wiki_fourth = wiki_first_type[1256:], wiki_second_type[1106:]

In [ ]:
# функция, получающая на вход дерево и возращающая предложение из этого дерева
def sentence_write(tree):
    node_forms = []
    nodes = tree.descendants
    for node in nodes:
        if node.ord == 1:
            node.form = node.form.title()
        elif node.ord > 1:
            if node.upos == 'PROPN':
                prop_name = node
                node.form = node.form.title()
                if prop_name.prev_node.deprel == 'amod':
                    prop_name.prev_node.form = prop_name.prev_node.form.title()
            else:

                node.form = node.form.lower()
        if node.ord == len(nodes)-1:
            node.misc = 'SpaceAfter=No'
        node_forms.append(node.form)

    sentence = ' '.join(node_forms)
    sentence = sentence.replace(' ,', ',')
    sentence = sentence.replace(' ?', '?')
    return sentence

С помощью функции из библиотеки numpy сэмплируем все вопросы, которые собираемся генерировать. При этом, тем типам, которые чаще встречаются в обучающей выборке, припишем бОльшие вероятности. Так, обработаем четыре группы вопросов

In [ ]:
yes_first_group = ['yes_first', 'yes_second', 'yes_third', 'yes_fourth', 
                   'yes_fifth', 'yes_sixth', 'yes_seventh', 'yes_eighth', 
                   'yes_ninth', 'yes_tenth', 'yes_eleventh', 'yes_twelfth',
                   'yes_twenty_fifth', 'yes_twenty_sixth', 'yes_twenty_seventh', 
                   'yes_twenty_eighth', 'yes_twenty_ninth', 'yes_thirtieth']
yes_first_probs = [0.25, 0.05, 0.005, 0.005, 0.0025, 0.0025, 0.25, 0.005, 0.005, 
                   0.005, 0.1, 0.1, 0.025, 0.025, 0.05, 0.05, 0.035, 0.035]

yes_second_group = ['yes_thirteenth', 'yes_fourteenth', 'yes_fifteenth', 
                    'yes_sixteenth', 'yes_seventeenth', 'yes_eighteenth', 
                    'yes_nineteenth', 'yes_twentieth', 'yes_twenty_first', 
                    'yes_twenty_second', 'yes_twenty_third', 'yes_twenty_fourth', 
                    'yes_thirty_first', 'yes_thirty_second', 'yes_thirty_third', 
                    'yes_thirty_fourth', 'yes_thirty_fifth', 'yes_thirty_sixth']
yes_second_probs = [0.25, 0.25, 0.0025, 0.0025, 0.005, 0.005, 0.05, 0.005, 0.005, 
                   0.005, 0.05, 0.05, 0.05, 0.05, 0.01, 0.01, 0.1, 0.1]

In [ ]:
import numpy as np

In [ ]:
questions_first_group = []
for sent in wiki_first:
  q_type = np.random.choice(yes_first_group, 1, p=yes_first_probs)
  if q_type == 'yes_first':
    tree = yes_first_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_second':
    tree = yes_second_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_third':
    tree = yes_third_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_fourth':
    tree = yes_fourth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_fifth':
    tree = yes_fifth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_sixth':
    tree = yes_sixth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_seventh':
    tree = yes_seventh_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_eighth':
    tree = yes_eighth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_ninth':
    tree = yes_ninth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_tenth':
    tree = yes_tenth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_eleventh':
    tree = yes_eleventh_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_twelfth':
    tree = yes_twelfth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_twenty_fifth':
    tree = yes_twenty_fifth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_twenty_sixth':
    tree = yes_twenty_sixth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_twenty_seventh':
    tree = yes_twenty_seventh_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_twenty_eighth':
    tree = yes_twenty_eighth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_twenty_ninth':
    tree = yes_twenty_ninth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)
  elif q_type == 'yes_thirtieth':
    tree = yes_thirtieth_type(sent) 
    sentence = sentence_write(tree)
    questions_first_group.append(sentence)

In [ ]:
len(questions_first_group)

1256

In [ ]:
with open("yes_questions_first_2023.txt", 'w', encoding='utf8') as yes_q_first:
  for elem in questions_first_group:
    yes_q_first.write(elem + '\n')

In [ ]:
questions_second_group = []
for sent in wiki_second:
  q_type = np.random.choice(yes_second_group, 1, p=yes_second_probs)
  if q_type == 'yes_thirteenth':
    tree = yes_thirteenth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_fourteenth':
    tree = yes_fourteenth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_fifteenth':
    tree = yes_fifteenth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_sixteenth':
    tree = yes_sixteenth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_seventeenth':
    tree = yes_seventeenth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_eighteenth':
    tree = yes_eighteenth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_nineteenth':
    tree = yes_nineteenth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_twentieth':
    tree = yes_twentieth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_twenty_first':
    tree = yes_twenty_first_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_twenty_second':
    tree = yes_twenty_second_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_twenty_third':
    tree = yes_twenty_third_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_twenty_fourth':
    tree = yes_twenty_fourth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_thirty_first':
    tree = yes_thirty_first_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_thirty_second':
    tree = yes_thirty_second_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_thirty_third':
    tree = yes_thirty_third_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_thirty_fourth':
    tree = yes_thirty_fourth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_thirty_fifth':
    tree = yes_thirty_fifth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)
  elif q_type == 'yes_thirty_sixth':
    tree = yes_thirty_sixth_type(sent) 
    sentence = sentence_write(tree)
    questions_second_group.append(sentence)

In [ ]:
len(questions_second_group)

1106

In [ ]:
with open("yes_questions_second_2023.txt", 'w', encoding='utf8') as yes_q_second:
  for elem in questions_second_group:
    yes_q_second.write(elem + '\n')

#### Генерация вопросов с ответом "нет"

In [ ]:
import numpy as np

In [ ]:
no_first_group = ['no_first', 'no_second', 'no_third', 'no_fourth', 
                  'no_fifth', 'no_sixth', 'no_seventh', 'no_eighth', 
                  'no_ninth', 'no_tenth', 'no_eleventh', 'no_twelfth',
                  'no_thirteenth', 'no_fourteenth', 'no_fifteenth', 
                  'no_sixteenth', 'no_seventeenth', 'no_eighteenth']
no_first_probs = [0.025, 0.15, 0.025, 0.025, 0.025, 0.05, 0.15, 0.025, 0.025, 
                  0.15, 0.025, 0.025, 0.025, 0.15, 0.025, 0.025, 0.025, 0.05]

no_second_group = ['no_nineteenth', 'no_twentieth', 'no_twenty_first', 
                   'no_twenty_second', 'no_twenty_third', 'no_twenty_fourth',
                   'no_twenty_fifth', 'no_twenty_sixth', 'no_twenty_seventh', 
                   'no_twenty_eighth', 'no_twenty_ninth', 'no_thirtieth', 
                   'no_thirty_first', 'no_thirty_second', 'no_thirty_third', 
                   'no_thirty_fourth', 'no_thirty_fifth', 'no_thirty_sixth']
no_second_probs = [0.025, 0.15, 0.025, 0.025, 0.025, 0.05, 0.15, 0.025, 0.025,
                   0.05, 0.025, 0.025, 0.025, 0.025, 0.15, 0.025, 0.15, 0.025]

In [ ]:
questions_third_group = []
for i, sent in enumerate(wiki_third):
  q_type = np.random.choice(no_first_group, 1, p=no_first_probs)
  if q_type == 'no_first':
    try:
        tree = no_first_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_second':
    try:
        tree = no_second_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_third':
    try:
        tree = no_third_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_fourth':
    try:
        tree = no_fourth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_fifth':
    try:
        tree = no_fifth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_sixth':
    try:
        tree = no_sixth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_seventh':
    try:
        tree = no_seventh_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_eighth':
    try:
        tree = no_eighth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_ninth':
    try:
        tree = no_ninth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_tenth':
    try:
        tree = no_tenth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_eleventh':
    try:
        tree = no_eleventh_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_twelfth':
    try:
        tree = no_twelfth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_thirteenth':
    try:
        tree = no_thirteenth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_fourteenth':
    try:
        tree = no_fourteenth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_fifteenth':
    try:
        tree = no_fifteenth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_sixteenth':
    try:
        tree = no_sixteenth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_seventeenth':
    try:
        tree = no_seventeenth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")
  elif q_type == 'no_eighteenth':
    try:
        tree = no_eighteenth_type(sent) 
        sentence = sentence_write(tree)
        questions_third_group.append(sentence)
    except:
        print(f"error occured on line {i}")

error occured on line 711
error occured on line 1258


In [ ]:
len(questions_third_group)

1257

In [ ]:
with open("no_questions_first_2023.txt", 'w', encoding='utf8') as no_q_first:
  for elem in questions_third_group:
    no_q_first.write(elem + '\n')

In [ ]:
questions_fourth_group = []
for i, sent in enumerate(wiki_fourth):
    q_type = np.random.choice(no_second_group, 1, p=no_second_probs)
    if q_type == 'no_nineteenth': 
        try:
            tree = no_nineteenth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twentieth':
        try:
            tree = no_twentieth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_first':
        try:
            tree = no_twenty_first_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_second':
        try:
            tree = no_twenty_second_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_third':
        try:
            tree = no_twenty_third_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_fourth':
        try:
            tree = no_twenty_fourth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_fifth':
        try:
            tree = no_twenty_fifth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_sixth':
        try:
            tree = no_twenty_sixth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_seventh':
        try:
            tree = no_twenty_seventh_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_eighth':
        try:
            tree = no_twenty_eighth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_twenty_ninth':
        try:
            tree = no_twenty_ninth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_thirtieth':
        try:
            tree = no_thirtieth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_thirty_first':
        try:
            tree = no_thirty_first_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_thirty_second':
        try:
            tree = no_thirty_second_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_thirty_third':
        try:
            tree = no_thirty_third_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_thirty_fourth':
        try:
            tree = no_thirty_fourth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_thirty_fifth':
        try:
            tree = no_thirty_fifth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")
    elif q_type == 'no_thirty_sixth':
        try:
            tree = no_thirty_sixth_type(sent) 
            sentence = sentence_write(tree)
            questions_fourth_group.append(sentence)
        except:
            print(f"error occured on line {i}")

error occured on line 1105


In [ ]:
len(questions_fourth_group)

1105

In [ ]:
with open("no_questions_second_2023.txt", 'w', encoding='utf8') as no_q_second:
  for elem in questions_fourth_group:
    no_q_second.write(elem + '\n')